In [1]:
import sys
# !{sys.executable} -m pip install --upgrade pip --user
# !{sys.executable} -m pip install h2o==3.24.0.5

In [1]:
import gc
# import resource
import datetime
import pandas as pd
import numpy as np
import h2o
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_colwidth', -1)

import os
cwd = os.getcwd()
print("current work director: ", cwd)
# cwd

import time

startcnt = time.time()
start = datetime.datetime.fromtimestamp(startcnt-14400).strftime('%c')
print("starting counter")
print(start)

current work director:  C:\Users\I25765\OneDrive - Verisk Analytics\Documents\Projects\RACA Symbol Liab Model\RACA_Prod\TTT_scp\AWS_TEST\SHAP
starting counter
Tue Aug 20 07:12:11 2019


In [2]:
h2o.init(min_mem_size='8g')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.211-b12, mixed mode)
  Starting server from C:\Anaconda\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\I25765\AppData\Local\Temp\tmptqtg90lg
  JVM stdout: C:\Users\I25765\AppData\Local\Temp\tmptqtg90lg\h2o_i25765_started_from_python.out
  JVM stderr: C:\Users\I25765\AppData\Local\Temp\tmptqtg90lg\h2o_i25765_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,2 months and 1 day
H2O cluster name:,H2O_from_python_i25765_w654jo
H2O cluster total nodes:,1
H2O cluster free memory:,7.667 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [13]:
# Generate SHAP value using h2o's built-in function predict_contributions()
startread = time.time()

shap_h2o = gbm_ttt_sXYY5.predict_contributions(train[x])

endread = time.time()
print("read time - training ds: ", round(endread - startread))

read time - training ds:  73


In [16]:
shap_h2o.shape

(253184, 57)

In [13]:
df_shap_h2o = shap_h2o

In [16]:
startread = time.time()

col_less_than_5 = []
for col in [x for x in shap_h2o.columns if x in cat]:
    if len(train[col].table()) <=5:
        col_less_than_5.append(col)
        
endread = time.time()
print("read time - training ds: ", round(endread - startread))

read time - training ds:  2


In [17]:
# Generate variable importance order
df_shap_value_order = df_shap_h2o.abs().mean(axis=0,return_frame=True).as_data_frame().drop('abs(BiasTerm)',axis=1)
col_name_abs = df_shap_value_order.stack().sort_values(ascending=False)[0].index.tolist()
#col_name_abs_updated = [x for x in col_name_abs if ('.n)' not in x and '.N)' not in x)]

In [18]:
#Get Top () variables
top = 15 ######## pleasure specify number of top variables
shap_value_order = [x[4:-1] for x in col_name_abs if x[4:-1] not in list(set(cat) - set(col_less_than_5))][:top]

In [19]:
shap_value_order

['cko_max_gvwcf',
 'cko_max_msrp',
 'cko_wheelbase_meanf',
 'cko_eng_disp_mean',
 'ENG_DISPLCMNT_CI',
 'cko_max_gvwc',
 'cko_wheelbase_mean',
 'cko_turbo_super',
 'TRK_REAR_AXL_CD',
 'cko_dtrl',
 'WHL_DRVN_CNTf',
 'SHIP_WGHT_LBS',
 'cko_weight_max',
 'TRK_BED_LEN_CD',
 'cko_fuel']

In [25]:
# Convert to Pandas Dataframe
startread = time.time()

df_shap = df_shap_h2o[shap_value_order].as_data_frame().iloc[filtered_index,]

endread = time.time()
print("read time - training ds: ", round(endread - startread))

read time - training ds:  4


In [27]:
# Convert to Pandas Dataframe
startread = time.time()

df = train[shap_value_order].as_data_frame().iloc[filtered_index,]

endread = time.time()
print("read time - training ds: ", round(endread - startread))

read time - training ds:  1


In [69]:
# Sample some data in order to speed up calculation
df_sampled = df#.sample(n=20000, random_state=1337)
df_shap_sampled = df_shap#.sample(n=20000, random_state=1337)

In [70]:
# Reshape for train dataset
reshape_df = df_sampled.stack(dropna=False).reset_index().iloc[:,1:]
reshape_df.columns = ['col_name', 'value']

# Reshape for SHAP dataset
reshape_df_shap = df_shap_sampled.stack(dropna=False).reset_index().iloc[:,1:]
reshape_df_shap.columns = ['col_name', 'shap_value']

#Join reshape_df and reshape_df_shap together
new_df = reshape_df.join(reshape_df_shap['shap_value'])

In [71]:
len(new_df)

1643310

In [119]:
# Sample some data in order to speed up calculation
new_df_sampled = new_df.sample(n=50000,random_state=1)

In [ ]:
# Generate Shap Chart
startread = time.time()

n_cols = len(shap_value_order)
#sns.set(font_scale=1.5) # Adjust the font size
fig, axs = plt.subplots(nrows=n_cols,figsize=(15,36))
fig.subplots_adjust(hspace=0.4)

i=0
for col in shap_value_order:
    
    if col not in cat:
        d = new_df_sampled[new_df_sampled['col_name'] == col]
        palette = sns.color_palette("coolwarm", n_colors=d['value'].nunique())
        g = sns.swarmplot(y=d['col_name'], ax=axs[i], x=d['shap_value'], hue=d['value'], palette=palette, dodge=False, size=2)
        g.legend_.remove()
    else:
        d = new_df_sampled[new_df_sampled['col_name'] == col]
        palette = sns.color_palette("bright", n_colors=d['value'].nunique())
        g = sns.swarmplot(y=d['col_name'], ax=axs[i], x=d['shap_value'], hue_order = list(d['value'].sort_values().unique()), hue=d['value'], palette=palette, dodge=False, size=2)
    axs[i].set_xlim(-0.5,0.5)
    axs[i].set(title=col)
    axs[i].set(ylabel = '')
    axs[i].set(xlabel = '')
    axs[i].get_yaxis().set_visible(False)
    i+=1

axs[n_cols-1].set(xlabel = 'SHAP value')
plt.savefig('SHAP_chart.png')

sns.palplot(sns.color_palette("coolwarm", 15))            
plt.xlabel('Low ------------------------------------------------------------------------ Feature Value ------------------------------------------------------------------------ High')
 
endread = time.time()
print("read time - training ds: ", round(endread - startread))

In [ ]:
h2o.cluster().shutdown()